# AiiDA

In [1]:
from aiida_workgraph import task, spec

In [2]:
from aiida import load_profile
load_profile()

Profile<uuid='3540adb9048947fcaafe2fc1cecf8bfa' name='pwd'>

In [3]:
@task(outputs=['prod', 'div'])
def get_prod_and_div(x, y):
    return {"prod": x * y, "div": x / y}

In [4]:
@task
def get_sum(x, y):
    return x + y

In [5]:
@task
def get_square(x):
    return x ** 2

In [6]:
@task.graph
def get_inner_part(a, b):
    prod_and_div = get_prod_and_div(x=a, y=b)
    return get_sum(x=prod_and_div.prod, y=prod_and_div.div).result

In [7]:
@task.graph
def get_total_workflow(c, d):
    tmp_sum = get_inner_part(a=c, b=d).result
    return get_square(x=tmp_sum).result

In [8]:
wg = get_total_workflow.build(c=1, d=2)
wg.run()

10/23/2025 09:04:01 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_inner_part
10/23/2025 09:04:01 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [3|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 4
10/23/2025 09:04:02 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_prod_and_div
10/23/2025 09:04:02 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4|WorkGraphEngine|update_task_state]: Task: get_prod_and_div, type: PYFUNCTION, finished.
10/23/2025 09:04:02 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_sum
10/23/2025 09:04:02 PM <99> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [4|WorkGraphEngine|update_task_state]: Task

{'result': <Float: uuid: 56164cdf-df06-4868-b205-acf9ad4376b9 (pk: 11) value: 6.25>}

# jobflow

In [9]:
from jobflow import job, Flow, run_locally

In [10]:
@job
def get_prod_and_div(x, y):
    return {"prod": x * y, "div": x / y}

In [11]:
@job
def get_sum(x, y):
    return x + y

In [12]:
@job
def get_square(x):
    return x ** 2

In [13]:
prod_and_div = get_prod_and_div(x=1, y=2)
tmp_sum = get_sum(x=prod_and_div.output.prod, y=prod_and_div.output.div)
inner_flow = Flow([prod_and_div, tmp_sum], output=tmp_sum.output)
result = get_square(x=inner_flow.output)
total_workflow = Flow([inner_flow, result])

In [14]:
run_locally(total_workflow)

2025-10-23 21:04:04,072 INFO Started executing jobs locally
2025-10-23 21:04:04,239 INFO Starting job - get_prod_and_div (66408a02-3f4c-4516-83d7-b64f98bf4d50)
2025-10-23 21:04:04,242 INFO Finished job - get_prod_and_div (66408a02-3f4c-4516-83d7-b64f98bf4d50)
2025-10-23 21:04:04,242 INFO Starting job - get_sum (97ed58b4-7c53-4aad-b506-ce9a658fbb97)
2025-10-23 21:04:04,244 INFO Finished job - get_sum (97ed58b4-7c53-4aad-b506-ce9a658fbb97)
2025-10-23 21:04:04,244 INFO Starting job - get_square (7b99113d-92c2-4113-b2b0-7a5cbcf126cb)
2025-10-23 21:04:04,245 INFO Finished job - get_square (7b99113d-92c2-4113-b2b0-7a5cbcf126cb)
2025-10-23 21:04:04,246 INFO Finished executing jobs locally


{'66408a02-3f4c-4516-83d7-b64f98bf4d50': {1: Response(output={'prod': 2, 'div': 0.5}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '97ed58b4-7c53-4aad-b506-ce9a658fbb97': {1: Response(output=2.5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 '7b99113d-92c2-4113-b2b0-7a5cbcf126cb': {1: Response(output=6.25, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))}}

# pyiron_base

In [15]:
from pyiron_base import job

In [16]:
@job(output_key_lst=['prod', 'div'])
def get_prod_and_div(x, y):
    return {"prod": x * y, "div": x / y}

In [17]:
@job
def get_sum(x, y):
    return x + y

In [18]:
@job
def get_square(x):
    return x ** 2

In [19]:
@job
def get_inner_part(a, b):
    prod_and_div = get_prod_and_div(x=1, y=2)
    return get_sum(x=prod_and_div.output.prod, y=prod_and_div.output.div).pull()

In [20]:
@job
def get_total_workflow(c, d):
    tmp_sum = get_inner_part(a=c, b=d)
    return get_square(x=tmp_sum).pull()

In [21]:
result = get_total_workflow(c=1, d=2)

In [22]:
result.pull()

The job get_total_workflow_0aaf08712456c7d3a49f7c3f01fdbfa6 was saved and received the ID: 1
The job get_inner_part_47b2c5618d08b12a4aed388c033c0f80 was saved and received the ID: 2
The job get_prod_and_div_e0304c807117683fb07505700833c1b2 was saved and received the ID: 3
The job get_sum_07da20f6280af1dd540bee904ca731eb was saved and received the ID: 4
The job get_square_3ab2091487de8071a940928bcb44c66b was saved and received the ID: 5


6.25